# Project to classify Beta Spindles

First load the data:

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
from sklearn.model_selection import train_test_split
from keras.models import Input,Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adamax
from keras.utils.np_utils import to_categorical

In [ ]:
#Function to load all the data
def load_all_data(filename):
    import numpy
    
    a = np.load(filename)
    d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))

    return d

In [ ]:
#Load the data
filename = '<FILENAME>' #fill in the path/filename you saved the data at
X = load_all_data(filename)[0]#I think this is the array order that I used
y = load_all_data(filename)[1]

IDs = load_all_data(filename)[2]
#we ignore age for now

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
test_size = 0.2
gss = GroupShuffleSplit(1, test_size)#you can look up specifics in the sklearn documentation

for train,test in gss.split(X, y, IDs):
    X_train = X[train]
    y_train = y[train]
    IDs_train = IDs[train]
    
    X_test = X[test]
    y_test = y[test]
    IDs_test = IDs[test]

fileoutname = '<FILEOUTNAME>'# fill in the path/filename you want to save the data at.
#Now save the train and test data
np.savez(fileoutname,X_train, y_train, X_test, y_test,IDs_train,IDs_test)

We dont look at the Test data again untill we are happy about the results of the model development and are ready to tests its generalizability to independent data.

You can use the same method to extract your development set

In [ ]:
for train,test in gss.split(X_train, y_train, IDs_train):
#etc fill in yourself :-)
#call the development set X_dev, y_dev, IDs_dev

You can use np.expand_dims to add an extra dimension at the end of the matrix (because the neural networks using 
Conv2D need matrices of samples x height x width x channels). Y needs to be transformed 'to categorical' for the output that we want. We don't only want to have 1 or 0 as output. We want to know how sure the model is about this, in this case.

In [ ]:
X_train2 = np.expand_dims(X_train2)#etc
y_train2 = to_categorical(y_train2)#etc --> same for the development X and y

Next would be to build a Convolutional Neural Network. Lets start with a (relatively) simple one.

In [ ]:
def simpleCNN(self, units = 10):
    
    import keras
    from keras.layers import Dense
    from keras.layers import Conv2D
    from keras.models import Model, Input
    
    inp =  Input(shape = self.shape[1:], name='inp')
    #layer 1
    x = Conv2D(units, kernel_size=(1,1), strides = (1,1), activation='relu', data_format='channels_last')(inp)  
    #layer 2
    x = Conv2D(units, kernel_size=(2,2), strides = (1,1), activation='relu', data_format='channels_last')(x)  
    #layer 3 
    x = Flatten()(x)
    #layer4
    out = Dense(2, activation='softmax',name='out')(x)
    
    model = Model(inputs = inp, outputs = out)
    
    return model

In [ ]:
from keras.optimizers import Adamax
adamax = Adamax(lr=3e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0); #3e-4; 2e-3 is a default.

model.compile(optimizer=adamax, loss='categorical_crossentropy', metrics=['accuracy', matthews_correlation])
model.fit(X_train2, y_train2, epochs=epochs, batch_size=batch_size, verbose = 1, validation_data = (X_dev, y_dev))

I suggest you start playing around with this. Read the sklearn and Keras documentation and try out several possibilities, e.g. more/less layers, dropout, different outputs, different optimizers. You could also choose to use less data if it takes to long for your computer to make the calculations. There are lots of things to play around with, so I look forward to see what you come up with!